In [ ]:
import networkx as nx
import numpy as np
import os
import io
import json
from multiprocessing import Process, Queue
import multiprocessing

In [87]:
# algorithm inspired by https://github.com/jeroenvldj/bow-tie_detection/blob/master/bow-tie_detection.py

def bowtie_analysis(G):
    # reverse all direction of the graph
    GT = nx.reverse(G, copy=True)
    # calculate SSC
    SSC = max(list(nx.strongly_connected_components(G)),key=len)    
    
    
    # take any node n from SSC and do a depth first search 
    # through directed graph beginning from node n
    v_any = list(SSC)[0]
    DFS_G = set(nx.dfs_tree(G,v_any).nodes())
    DFS_GT = set(nx.dfs_tree(GT,v_any).nodes())
    OUT = DFS_G - SSC
    IN = DFS_GT - SSC
    V_rest = set(G.nodes()) - SSC - OUT - IN

    TUBES = set()
    INTENDRILS = set()
    OUTTENDRILS = set()
    OTHER = set()

    for v in V_rest:
        # irv => in reaches node v
        irv = len(IN & set(nx.dfs_tree(GT,v).nodes())) is not 0
        # vro => node v reaches out
        vro = len(OUT & set(nx.dfs_tree(G,v).nodes())) is not 0
        if irv and vro:
            TUBES.add(v)
        elif irv and not vro:
            INTENDRILS.add(v)
        elif not irv and vro:
            OUTTENDRILS.add(v)
        elif not irv and not vro:
            OTHER.add(v)

    FRINGE = set()
    DISCONNECTED = set()
    for o in OTHER:
        # orIT => node o reaches INTENDRILS  
        orIT = len(INTENDRILS & set(nx.dfs_tree(G,o))) is not 0
        # OTro => OUTTERNDIRLS reaches node o
        OTro = len(OUTTENDRILS & set(nx.dfs_tree(GT,o))) is not 0
        if orIT or OTro:
            FRINGE.add(o)
        else:
            DISCONNECTED.add(o)
    
    TENDRILS = INTENDRILS.union(OUTTENDRILS)
    
    def component_result(name, graph_nodes):
        return{ name        : len(graph_nodes),
                name + "_s" : sum([G.nodes()[node]["is_sink"] for node in graph_nodes]),
                name + "_m" : sum([G.nodes()[node]["is_miner"] for node in graph_nodes])}

    result_dict = dict()
    result_dict.update(component_result("nodes", G.nodes()))
    result_dict.update(component_result("ssc", SSC))
    result_dict.update(component_result("in", IN))
    result_dict.update(component_result("out",OUT))
    result_dict.update(component_result("tubes", TUBES))
    result_dict.update(component_result("tendrils", TENDRILS))
    result_dict.update(component_result("fringe", FRINGE))
    result_dict.update(component_result("disconnected", DISCONNECTED))
    
    return result_dict

In [88]:
def files_walker(files, json_output,directory):
    d = {}
    #files = os.listdir(directory)
    #files = directory
    for file in files:
        with open(directory+'/'+file, 'r', encoding='utf8', errors='ignore') as f:
            G = nx.read_graphml(f)
            bowtie_dict = bowtie_analysis(G)
            with open(json_output, "r+") as fi:
                data = json.load(fi)
                new_entry = {file[:-8] : bowtie_dict}
                data.update(new_entry)
                fi.seek(0)
                json.dump(data, fi, indent=4)
    return True

In [107]:
nr_cores = multiprocessing.cpu_count()
print(nr_cores)
import time
start_time = time.time()

core_dict = {}
np_files = np.array(os.listdir("data/NET-btc-heur_0-week"))
directory = "data/NET-btc-heur_0-week"
#chunk_lst = np.array_split(np_files, nr_cores)
chunk_lst = np_files

for i in range(4):
    process_name = f"p{i}"
    file_name = f"test{i}.json"

    with io.open(os.path.join(file_name), 'w') as db_file:
        db_file.write(json.dumps({}))
    
    
    core_dict[process_name] = Process(target = files_walker, args=(([chunk_lst[i]],file_name, directory)))
    print("proccess added")

for key, item in core_dict.items():
    item.start()
    print("started process: {}".format(key))

for key, item in core_dict.items():
    item.join()
    print("joined process: {}".format(key))
    
print("--- %s seconds ---" % (time.time() - start_time))

8
proccess added
proccess added
proccess added
proccess added
started process: p0
started process: p1
started process: p2
started process: p3
joined process: p0
joined process: p1
joined process: p2
joined process: p3
--- 42.60962653160095 seconds ---


In [102]:
core_dict = {}
for key, item in core_dict.items():
    print(2)

In [84]:
np_files = np.array(os.listdir("data/NET-btc-heur_0-week"))
directory = "data/NET-btc-heur_0-week"
chunk_lst = np.array_split(np_files, )
chunk_lst

[array(['2010-09-06.graphml', '2012-04-23.graphml', '2013-09-16.graphml'],
       dtype='<U18'),
 array(['2012-04-30.graphml', '2009-01-26.graphml', '2012-10-08.graphml'],
       dtype='<U18'),
 array(['2012-01-30.graphml', '2012-06-04.graphml'], dtype='<U18'),
 array(['2012-12-31.graphml', '2010-10-18.graphml'], dtype='<U18'),
 array(['2009-05-04.graphml', '2010-04-26.graphml'], dtype='<U18'),
 array(['2013-04-15.graphml', '2010-02-08.graphml'], dtype='<U18'),
 array(['2011-10-31.graphml', '2011-10-17.graphml'], dtype='<U18'),
 array(['2013-07-08.graphml', '2011-11-28.graphml'], dtype='<U18')]

In [96]:
np_files = np.array(os.listdir("data/NET-btc-heur_0-week"))
directory = "data/NET-btc-heur_0-week"
#chunk_lst = np.array_split(np_files, nr_cores)
chunk_lst = np_files
[chunk_lst[i]]


['2010-09-06.graphml']

In [93]:
with open(directory+'/'+chunk_lst[3][3], 'r', encoding='utf8', errors='ignore') as f:
    G = nx.read_graphml(f)

In [100]:
i=0
process_name = f"p{i}"
process_name

'p0'

In [102]:
process_name = f"test{i}.json"
process_name

'test0.json'